In [1]:
import fastjet
import uproot
import awkward as ak
import hist
import numpy as np
import vector
vector.register_awkward()

In [2]:
def remove_muons(reco):
    '''Remove muons in a silly way:
    Remove any reco particle in the mass range 0.10566 +/- 0.03
    '''
    err = 0.03
    ll = reco.mass <= abs(0.10566-err)
    hl = reco.mass >= abs(0.10566+err)
    ll = ak.drop_none(ll)
    hl = ak.drop_none(hl)
    return reco[ll | hl]

In [3]:
redirector = ''
''' In case of problems, Download the root files from the following links:
https://prayag.web.cern.ch/share/FCC/eos/experiment/fcc/ee/generation/DelphesEvents/winter2023/IDEA/wzp6_ee_mumuH_Hbb_ecm240/events_159112833.root
'''

events_path = redirector+'eos/experiment/fcc/ee/generation/DelphesEvents/winter2023/IDEA/wzp6_ee_mumuH_Hbb_ecm240/events_159112833.root:events'

def _remove_not_interpretable(branch, emit_warning=True):
    if isinstance(
        branch.interpretation, uproot.interpretation.identify.uproot.AsGrouped
    ):
        for name, interpretation in branch.interpretation.subbranches.items():
            if isinstance(
                interpretation, uproot.interpretation.identify.UnknownInterpretation
            ):
                if emit_warning:
                    warnings.warn(
                        f"Skipping {branch.name} as it is not interpretable by Uproot"
                    )
                return False
    if isinstance(
        branch.interpretation, uproot.interpretation.identify.UnknownInterpretation
    ):
        if emit_warning:
            warnings.warn(
                f"Skipping {branch.name} as it is not interpretable by Uproot"
            )
        return False

    try:
        _ = branch.interpretation.awkward_form(None)
    except uproot.interpretation.objects.CannotBeAwkward:
        if emit_warning:
            warnings.warn(
                f"Skipping {branch.name} as it is it cannot be represented as an Awkward array"
            )
        return False
    else:
        return True

events = uproot.open(
    events_path,
    open_files=False,
    filter_name =  lambda x : "PARAMETERS" not in x,
    filter_branch = _remove_not_interpretable
)

In [4]:
Reco = ak.zip(
    {
        'E':events['ReconstructedParticles/ReconstructedParticles.energy'].arrays()['ReconstructedParticles.energy'],
        'px':events['ReconstructedParticles/ReconstructedParticles.momentum.x'].arrays()['ReconstructedParticles.momentum.x'],
        'py':events['ReconstructedParticles/ReconstructedParticles.momentum.y'].arrays()['ReconstructedParticles.momentum.y'],
        'pz':events['ReconstructedParticles/ReconstructedParticles.momentum.z'].arrays()['ReconstructedParticles.momentum.z'],
    },
    with_name='Momentum4D'
)
              

pseudo_jets = remove_muons(Reco)

#Ensure there are at least 2 reco particles before clustering
pseudo_jets = pseudo_jets[ak.num(pseudo_jets, axis=1) > 1]

## Ignoring the stuff above, we now have pseudojets available to play around

In [5]:
class E0_scheme:
    '''
    E0 Scheme Recombiner
    Find the C++ equivalent at https://github.com/HEP-FCC/FCCAnalyses/blob/master/addons/FastJet/src/ExternalRecombiner.cc
    '''
    def preprocess(p):
        pass
        
    def recombine(pa, pb):
    
        psum = pa+pb
        pmag = np.sqrt(psum.px()**2 + psum.py()**2 + psum.pz()**2)
        E0scale = psum.E() / pmag

        psum.reset(
                psum.px()*E0scale,
                psum.py()*E0scale,
                psum.pz()*E0scale,
                psum.E()
        )
        return psum

In [6]:
def get_jets(pseudojets):
    
    jetdef = fastjet.JetDefinition0Param(fastjet.ee_kt_algorithm)
    jetdef.set_python_recombiner(E0_scheme)

    cluster = fastjet.ClusterSequence(pseudo_jets, jetdef)
    
    return cluster.exclusive_jets(2), cluster.exclusive_jets_constituents(2)

In [7]:
# fastjet.JetDefinition0Param(fastjet.ee_kt_algorithm, fastjet.E_scheme)

In [8]:
# jets, jet_constituents = get_jets(pseudo_jets)

In [9]:
jetdef = fastjet.JetDefinition0Param(fastjet.ee_kt_algorithm)
jetdef.set_python_recombiner(E0_scheme)

In [10]:
def make_fj_ps(ak_array):
    px = ak_array.px
    py = ak_array.py
    pz = ak_array.pz
    E = ak_array.E

    total = ak.num(ak_array, axis=0)

    out = []
    for i in range(total):
        out.append(
            fastjet.PseudoJet(
                float(px[i]),
                float(py[i]),
                float(pz[i]),
                float(E[i])
            )
        )
    return out

In [11]:
def get_jets(n_events):
    jets = []
    
    entry_stop = n_events
    
    reduced_pseudo_jets = ak.values_astype(pseudo_jets[:entry_stop], 'float16')
    for ev in reduced_pseudo_jets:
        cluster = fastjet.ClusterSequence(make_fj_ps(ev), jetdef)
        jets.append(cluster.exclusive_jets(2))
    return jets

In [12]:
from concurrent.futures import ThreadPoolExecutor

In [ ]:
with ThreadPoolExecutor(max_workers=8) as executor:
    future = executor.submit(get_jets, 100000)
    print(future.result())

#--------------------------------------------------------------------------
#                         FastJet release 3.4.3
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the GNU GPL v2 or higher.  
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code,
# CGAL and 3rd party plugin jet algorithms. See COPYING file for details.
#--------------------------------------------------------------------------


In [ ]:
# cluster = fastjet.ClusterSequence(pseudo_jets, jetdef)

# jets = cluster.exclusive_jets(2)

# jet_constituents = cluster.exclusive_jets_constituents(2)

# del cluster
# del jetdef

dijets = ak.sum(jets, axis=1)

h = hist.Hist.new.Reg(100,0,200).Double().fill(dijets.mass)

In [ ]:
h